In [1]:
import pandas as pd 
from feature_selection import *
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from dataset import make_dataset
from os.path import join
from tensorflow import keras

In [49]:
train = pd.read_parquet('./jeju_data/train_new.parquet')
test = pd.read_parquet('./jeju_data/test_new.parquet')

In [50]:
def weeeek(train):
    Mon_train = train[train['day_of_week']=='월']
    Tus_train = train[train['day_of_week']=='화']
    Wed_train = train[train['day_of_week']=='수']
    Thu_train = train[train['day_of_week']=='목']
    Fri_train = train[train['day_of_week']=='금']
    Sat_train = train[train['day_of_week']=='토']
    Sun_train = train[train['day_of_week']=='일']

    return Mon_train, Tus_train, Wed_train, Thu_train, Fri_train, Sat_train, Sun_train

In [51]:
Mon_train, Tus_train, Wed_train, Thu_train, Fri_train, Sat_train, Sun_train = weeeek(train)
Mon_test, Tus_test, Wed_test, Thu_test, Fri_test, Sat_test, Sun_test = weeeek(test)

In [52]:
def le(train, test):
    str_col = ['day_of_week', 'start_turn_restricted', 'end_turn_restricted']

    for i in str_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]):
            if label not in le.classes_:
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i])

    X = train.drop(
    ['id', 'base_date', 'target', 'road_name', 'start_node_name', 'end_node_name', 'vehicle_restricted'], axis=1)

    y = train['target']

    test = test.drop(

        ['id', 'base_date', 'road_name', 'start_node_name', 'end_node_name', 'vehicle_restricted'], axis=1)

    return X, y, test

In [53]:
Mon_x, Mon_y, Mon_test = le(Mon_train, Mon_test)
Tus_x, Tus_y, Tus_test = le(Tus_train, Tus_test)
Wed_x, Wed_y, Wed_test = le(Wed_train, Wed_test)
Thu_x, Thu_y, Thu_test = le(Thu_train, Thu_test)
Fri_x, Fri_y, Fri_test = le(Fri_train, Fri_test)
Sat_x, Sat_y, Sat_test = le(Sat_train, Sat_test)
Sun_x, Sun_y, Sun_test = le(Sun_train, Sun_test)

C:\Users\jkpark\AppData\Local\Temp\ipykernel_3672\4018098574.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i] = le.transform(train[i])
C:\Users\jkpark\AppData\Local\Temp\ipykernel_3672\4018098574.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[i] = le.transform(test[i])
C:\Users\jkpark\AppData\Local\Temp\ipykernel_3672\4018098574.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [54]:
Mon_test

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
6,0,4,1,107,0,0,60.0,0.0,0.0,0,33.498363,126.516407,0,33.499973,126.516469,1
10,0,4,3,103,0,0,70.0,0.0,0.0,0,33.499973,126.516469,1,33.499848,126.519872,1
11,0,10,1,103,0,0,70.0,0.0,0.0,0,33.494763,126.533654,0,33.492718,126.535597,0
15,0,6,3,103,0,0,70.0,0.0,0.0,0,33.480679,126.472216,1,33.478257,126.468081,1
17,0,8,2,107,0,0,60.0,0.0,0.0,0,33.249949,126.505664,0,33.252183,126.506069,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291189,0,11,1,106,0,0,60.0,43200.0,0.0,3,33.291747,126.505010,0,33.291731,126.504396,0
291195,0,16,1,106,0,0,60.0,0.0,0.0,0,33.291404,126.513613,0,33.291747,126.505010,0
291198,0,16,1,106,0,0,60.0,43200.0,0.0,3,33.291404,126.513613,0,33.291359,126.514457,0
291223,0,11,3,103,0,0,50.0,0.0,0.0,3,33.515501,126.557975,0,33.514390,126.555913,0


In [55]:
from xgboost import XGBRegressor

def xgb_model(x, y, test):
    model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
    model = model.fit(x, y)
    y_pred = model.predict(test)

    return y_pred

In [56]:
Mon_y_pred = xgb_model(Mon_x, Mon_y, Mon_test)
Tus_y_pred = xgb_model(Tus_x, Tus_y, Tus_test)
Wed_y_pred = xgb_model(Wed_x, Wed_y, Wed_test)
Thu_y_pred = xgb_model(Thu_x, Thu_y, Thu_test)
Fri_y_pred = xgb_model(Fri_x, Fri_y, Fri_test)
Sat_y_pred = xgb_model(Sat_x, Sat_y, Sat_test)
Sun_y_pred = xgb_model(Sun_x, Sun_y, Sun_test)

In [57]:
Mon_test['predict'] = Mon_y_pred
Tus_test['predict'] = Tus_y_pred
Wed_test['predict'] = Wed_y_pred
Thu_test['predict'] = Thu_y_pred
Fri_test['predict'] = Fri_y_pred
Sat_test['predict'] = Sat_y_pred
Sun_test['predict'] = Sun_y_pred

In [92]:
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission

,id,target
0,TEST_000000,0
1,TEST_000001,0
2,TEST_000002,0
3,TEST_000003,0
4,TEST_000004,0
...,...,...
291236,TEST_291236,0
291237,TEST_291237,0
291238,TEST_291238,0
291239,TEST_291239,0


In [105]:
def update_test(test, sample_submission):
    for i in range(len(test)):

        test_index = test.index[i]
        predict = test.loc[test_index]["predict"]
        sample_submission.loc[test_index, "target"] = predict
        
    return sample_submission

In [106]:
test_list = [Mon_test, Tus_test, Wed_test, Thu_test, Fri_test, Sat_test, Sun_test]
for i in test_list:
    sample_submission = update_test(i, sample_submission)
print(sample_submission)

                 id     target
0       TEST_000000  23.576605
1       TEST_000001  45.058136
2       TEST_000002  64.913765
3       TEST_000003  37.736603
4       TEST_000004  38.372349
...             ...        ...
291236  TEST_291236  48.224007
291237  TEST_291237  52.526199
291238  TEST_291238  21.936792
291239  TEST_291239  24.951241
291240  TEST_291240  45.065929

[291241 rows x 2 columns]


In [109]:
sample_submission.to_csv("model_7.csv", index=False)